# ⚙️ Python Fundamentals — 07 Context Managers (Async Example)

In modern Python frameworks like **FastAPI**, context managers are often used to handle **setup and cleanup** automatically.

Here we use an **async context manager** (`@asynccontextmanager`) to prepare and close resources when an app starts and stops.

## 1️⃣ Example: `@asynccontextmanager` with FastAPI

When your FastAPI app starts, you might want to create connections, memory objects, or models — and clean them up when the app stops.

This is what a **lifespan context** does:

In [2]:
from contextlib import asynccontextmanager
from fastapi import FastAPI

# Imagine these are your resources (databases, models, etc.)
graphs = {}

def build_graph(checkpointer=None):
    # Fake function: creates something that we store
    return {"checkpointer": checkpointer or "MemorySaver"}

class Settings:
    sqlite_path = "database.sqlite"

settings = Settings()

@asynccontextmanager
async def lifespan(app: FastAPI):
    # Called when app starts
    print("Starting app — creating resources...")
    graphs["temporary"] = build_graph()
    graphs["persistent"] = build_graph(checkpointer=settings.sqlite_path)

    # Everything before 'yield' runs at startup
    yield
    # Everything after 'yield' runs at shutdown
    print("Shutting down app — cleaning up resources...")
    graphs.clear()

# Attach lifespan to the FastAPI app
app = FastAPI(title="Chatbot Backend Example", lifespan=lifespan)

print("App ready with custom lifespan.")

App ready with custom lifespan.


### 🧠 Explanation (in plain English)
- `@asynccontextmanager` lets you define code that runs **before** and **after** an app’s life cycle.
- Everything before `yield` happens when the app **starts** (setup).
- Everything after `yield` happens when the app **stops** (cleanup).
- This is perfect for opening/closing databases, loading models, or building graphs (like LangChain or LangGraph).

FastAPI calls this function automatically because we attach it as `lifespan=lifespan` when creating the app.

### 🧭 Summary
- Use `@asynccontextmanager` for setup + teardown in async apps.
- Attach it to FastAPI with `lifespan=...` to control startup/shutdown logic.
- Keep it minimal: prepare before `yield`, clean after `yield`.

> It’s an elegant way to make sure your app starts and stops safely — no manual cleanup needed.